In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.agents import tool
from typing import Optional
from pydantic import BaseModel, Field
from langchain.tools.render import format_tool_to_openai_function
from dotenv import load_dotenv
import os
import json
import langchain

In [6]:
langchain.__version__

'0.1.19'

In [8]:
load_dotenv()
api_key=os.environ.get("OPENAI_API_KEY")

In [42]:
from openai import OpenAI

ImportError: cannot import name 'OpenAI' from 'openai' (/home/grace-nyutu/Documents/Redash_chatbot_add-on/venv/lib/python3.12/site-packages/openai/__init__.py)

In [40]:
client = openai(api_key=api_key)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who is the first president of India?"},
        {"role": "assistant", "content": "Rajendra Prasad is the first president of India."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response)


InvalidRequestError: Unrecognized request argument supplied: messages

In [16]:
extraction_functions = [
    {
        "name": "extract_information",
        "description": "extracts information",
        "parameters": {
            "type": "object",
            "properties": {
                "metric": {
                    "type": "string",
                    "description": "main metric we need to calculate, for example, 'number of users' or 'number of sessions'",
                },
                "filters": {
                    "type": "string",
                    "description": "filters to apply to the calculation (do not include filters on dates here)",
                },
                "dimensions": {
                    "type": "string",
                    "description": "parameters to split your metric by",
                },
                "period_start": {
                    "type": "string",
                    "description": "the start day of the period for a report",
                },
                "period_end": {
                    "type": "string",
                    "description": "the end day of the period for a report",
                },
                "output_type": {
                    "type": "string",
                    "description": "the desired output",
                    "enum": ["number", "visualisation"]
                }
            },
            "required": ["metric"],
        },
    }
]

In [18]:
# create a class inheriting from the BaseModel class and define all the fields

from pydantic import BaseModel, Field
from typing import Optional

class RequestStructure(BaseModel):
  """extracts information"""
  metric: str = Field(description = "main metric we need to calculate, for example, 'number of users' or 'number of sessions'")
  filters: Optional[str] = Field(description = "filters to apply to the calculation (do not include filters on dates here)")
  dimensions: Optional[str] = Field(description = "parameters to split your metric by")
  period_start: Optional[str] = Field(description = "the start day of the period for a report")
  period_end: Optional[str] = Field(description = "the end day of the period for a report")
  output_type: Optional[str] = Field(description = "the desired output", enum = ["number", "visualisation"])


# use LangChain to convert the Pydantic class into the OpenAI function
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

extract_info_function = convert_pydantic_to_openai_function(RequestStructure, 
    name = 'extract_information')

/home/grace-nyutu/Documents/Redash_chatbot_add-on/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [19]:
model = ChatOpenAI(temperature=0.1, model = 'gpt-3.5-turbo-1106')\
  .bind(functions = [extract_info_function])

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information from the provided request. \
            Extract ONLY the information presented in the initial request. \
            Don't add anything else. \
            Return partial information if something is missing."),
    ("human", "{request}")
])

extraction_chain = prompt | model

In [20]:
extract_info_function

{'name': 'extract_information',
 'description': 'extracts information',
 'parameters': {'properties': {'metric': {'description': "main metric we need to calculate, for example, 'number of users' or 'number of sessions'",
    'type': 'string'},
   'filters': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
    'description': 'filters to apply to the calculation (do not include filters on dates here)'},
   'dimensions': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
    'description': 'parameters to split your metric by'},
   'period_start': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
    'description': 'the start day of the period for a report'},
   'period_end': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
    'description': 'the end day of the period for a report'},
   'output_type': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
    'description': 'the desired output',
    'enum': ['number', 'visualisation']}},
  'required': ['metric',
   'filters',
   'dimensions',

Chatbot memory